<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_streaming_kafka_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the required kafka packages

In [15]:
output_path = '/content/drive/MyDrive/Colab Notebooks/data_engineering'

In [16]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.1 MB/s eta 0:00:00


### Import packages

In [17]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError


## Download and setup Kafka and Zookeeper instances

For demo purposes, the following instances are setup locally:

- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)


In [1]:
!curl -sSOL https://downloads.apache.org/kafka/3.7.0/kafka_2.12-3.7.0.tgz
!tar -xzf kafka_2.12-3.7.0.tgz

Kafka with defaults

In [2]:
!./kafka_2.12-3.7.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.0/config/zookeeper.properties
!./kafka_2.12-3.7.0/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.0/config/server.properties
!echo "Give the processes 10 seconds to start before proceeding."
!sleep 10

Give the processes 10 seconds to start before proceeding.


Is Kafka running?

In [7]:
!ps -ef | grep java

root        6474       1  1 22:14 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:
root        6884       1  5 22:14 ?        00:00:09 java -Xmx1G -Xms1G -server -XX:+UseG1GC -XX:MaxG
root        7658    5951  0 22:17 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        7660    7658  0 22:17 ?        00:00:00 grep java


Create the kafka topics with the following specs:

- sample-streaming-data: partitions=1

In [8]:
!./kafka_2.12-3.7.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic sample-streaming-data

Created topic sample-streaming-data.


Describe the topic for details on the configuration

In [9]:
!./kafka_2.12-3.7.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic sample-streaming-data

Topic: sample-streaming-data	TopicId: GOJvkxByQXKihHS9m94DQQ	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: sample-streaming-data	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


## generator python script

In [10]:
%%writefile generator.py

import sys
args = sys.argv  # a list of the arguments provided (str)
print("running generator.py", args)
iterations = int(args[1])
print(f'iterations: {iterations}')

def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  from kafka import KafkaProducer

  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8'), partition=0).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

import random
from time import sleep

def generate_data(rows=2):

  for i in range(1,rows):

    index_num = random.randint(0,1000000)
    print(index_num)
    keys = list([f'{index_num}'])
    msg = list([f'hello world!{index_num}'])
    data = zip(keys , msg)

  return data

for i in range(iterations):
  write_to_kafka("sample-streaming-data", generate_data())
  sleep(random.randint(0,10))



Writing generator.py


# retrieve messages

In [19]:
def retrieve_messages(topic='sample-streaming-data',bootstrap_servers = '127.0.0.1:9092',output_path='output_path'):

  from kafka import KafkaConsumer
  from kafka.structs import TopicPartition
  import gspread

  consumer = KafkaConsumer(
      bootstrap_servers=bootstrap_servers,
      auto_offset_reset='earliest',
      group_id='retrieve_last_items_group'
      )###,enable_auto_commit=True)
  # Read the specified partition
  consumer.assign([TopicPartition(topic, 0)])



  for msg in consumer:
      # Open the file in write mode ('a')
      file_path = 'steaming_data.csv'  # Replace with the desired file path
      file = open(file_path, 'a')

      print(f"the msg: {msg.value.decode('utf-8')}")
        # Write content to the file
      file.write(f"{msg.value.decode('utf-8')}\n")
      # Close the file
      file.close()

# write some data

In [12]:
%%script bash --bg

python generator.py 3

function for retrieve messages.

In [20]:
retrieve_messages()

KeyboardInterrupt: 

In [ ]:
!cat steaming_data.csv

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(9092)"))

In [ ]:
#%%write_file recommender.py

import numpy as np

# Sample user-item matrix
user_item_matrix = np.array([
    [4, 5, 0, 5, 0],
    [5, 0, 4, 0, 3],
    [0, 3, 0, 4, 5],
    [4, 0, 5, 0, 4],
    [0, 4, 0, 3, 0]
])

# Calculate item similarities using cosine similarity
def calculate_item_similarities(matrix):
    num_items = matrix.shape[1]
    similarities = np.zeros((num_items, num_items))

    for i in range(num_items):
        for j in range(num_items):
            if i != j:
                item_i = matrix[:, i]
                item_j = matrix[:, j]
                similarity = np.dot(item_i, item_j) / (np.linalg.norm(item_i) * np.linalg.norm(item_j))
                similarities[i, j] = similarity

    return similarities

# Generate item recommendations for a target user
def generate_item_recommendations(user_id, matrix, similarities, top_k=3):
    user_ratings = matrix[user_id]
    num_items = matrix.shape[1]

    item_scores = np.zeros(num_items)

    for i in range(num_items):
        if user_ratings[i] == 0:
            item_i_similarity = similarities[i]
            item_i_ratings = matrix[:, i]
            item_scores[i] = np.dot(item_i_similarity, item_i_ratings) / np.sum(item_i_similarity)
            print(item_scores)

    #item_scores = #np.argsort(item_scores)[::-1][:top_k]
    print(item_scores)

    return item_scores

# Calculate item similarities
item_similarities = calculate_item_similarities(user_item_matrix)

# Generate recommendations for user 2
user_id = 4
recommendations = generate_item_recommendations(user_id, user_item_matrix, item_similarities, top_k=6)

print(f"Recommendations for user {user_id}: {recommendations} with matrix {user_item_matrix[user_id]}")

In [ ]:
np.nonzero(recommendations)

In [ ]:
np.take(recommendations, np.nonzero(recommendations), axis=0)

In [ ]:
import numpy as np
arr1 = np.nonzero(recommendations)
arr2 = np.take(recommendations, np.nonzero(recommendations), axis=0)

In [ ]:
arr1inds = np.take(recommendations, np.nonzero(recommendations), axis=0).argsort()
sorted_arr1 = arr1[arr1inds[::-1]]
sorted_arr2 = arr2[arr1inds[::-1]]

In [ ]:
sorted_arr1

In [ ]:
from kafka import KafkaAdminClient, KafkaProducer
from kafka.admin import NewTopic

# Kafka broker details
bootstrap_servers = 'localhost:9092'

# Topic details
topic_name = 'my_topic'
num_partitions = 1
replication_factor = 1

# Create Kafka topic
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
topic = NewTopic(name=topic_name, num_partitions=num_partitions, replication_factor=replication_factor)
admin_client.create_topics([topic])

# Produce messages to Kafka topic
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
messages = ['Hello', 'World', 'Kafka', 'Python']

for message in messages:
    producer.send(topic_name, value=message.encode('utf-8'))

# Close Kafka producer
producer.flush()
producer.close()

In [ ]:
from kafka import KafkaConsumer

# Kafka broker details
bootstrap_servers = 'localhost:9092'

# Topic details
topic_name = 'my_topic'

# Kafka consumer configuration
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=bootstrap_servers,
    group_id='my_consumer_group2',
    auto_offset_reset='earliest',
    #enable_auto_commit=True,
    value_deserializer=lambda x: x.decode('utf-8')
)

# Consume and process messages from Kafka topic
for message in consumer:
    print("Received message:", message.value)

# Close Kafka consumer
consumer.close()
